In [51]:
import paddle
from paddle.vision.transforms import Normalize

def get_MNIST_dataloader():
    # 定义图像归一化处理方法，这里的CHW指图像格式需为 [C通道数，H图像高度，W图像宽度]
    transform = Normalize(mean=[127.5], std=[127.5], data_format='chw')
    # 下载数据集并初始化 dataset
    train_dataset = paddle.vision.datasets.MNIST(mode='train', transform=transform)
    test_dataset = paddle.vision.datasets.MNIST(mode='test', transform=transform)

    # 定义并初始化数据读取器
    train_loader = paddle.io.DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=1, drop_last=True)
    test_loader = paddle.io.DataLoader(test_dataset, batch_size=129, shuffle=False, num_workers=1, drop_last=False)

    return train_loader, test_loader

In [52]:
train_loader, test_loader = get_MNIST_dataloader()

In [48]:
import visualdl
import visualdl.server.app 
visualdl.server.app.run('./log',
                        host="127.0.0.1",
                        port=8080,
                        cache_timeout=20)

20556

In [24]:
# 定义模型结构
import paddle.nn.functional as F
from paddle.nn import Conv2D, MaxPool2D, Linear

# 多层卷积神经网络实现
class MNIST_CONV_INIT(paddle.nn.Layer):
    def __init__(self):
        super(MNIST_CONV_INIT, self).__init__()

        # 定义卷积层，输出特征通道20，卷积核大小5
        self.conv1 = Conv2D(in_channels=1, out_channels=20, kernel_size=5, stride=1, padding=2)
        # 定义池化层，池化核的大小为2，步长为2
        self.max_pool1 = MaxPool2D(kernel_size=2, stride=2)
        # 定义卷积层，输出特征20维，卷积核大小为5
        self.conv2 = Conv2D(in_channels=20, out_channels=20, kernel_size=5, stride=1, padding=2)
        # 定义池化层
        self.max_pool2 = MaxPool2D(kernel_size=2, stride=2)
        # 定义一层全连接层，输入维度是10
        self.fc = Linear(in_features=980, out_features=10)

    # 定义前向，卷积后使用池化，然后全连接输出
    # 激活函数使用ReLU，全连接层使用softmax
    def forward(self, inputs, label):
        x = self.conv1(inputs)
        x = F.relu(x)
        x = self.max_pool1(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = self.max_pool2(x)
        x = paddle.reshape(x, [x.shape[0], 980])
        x = self.fc(x)
        if label is not None:
            acc = paddle.metric.accuracy(input=x, label=label)
            return x, acc
        else:
            return x
use_gpu = True
paddle.set_device('gpu:0') if use_gpu else paddle.set_device('cpu')




Place(gpu:0)

In [25]:
# 定义训练过程
from visualdl import LogWriter
log_writer = LogWriter(logdir="./log")
def train_MNIST_INIT(model):

    # 定义优化器
    opt = paddle.optimizer.Adam(learning_rate=0.003, parameters=model.parameters())

    # 训练参数
    EPOCH_NUM = 5
    iter = 0
    iter2 = 0
    for epoch_id in range(EPOCH_NUM):
        for batch_id, data in enumerate(train_loader()):
            model.train()
            images, labels = data 
            images = paddle.to_tensor(images)
            labels = paddle.to_tensor(labels)

            # 前向
            predicts, acc = model(images, labels)

            # 损失
            loss = F.cross_entropy(predicts, labels)
            avg_loss = paddle.mean(loss)
            

            # 记录损失
            if batch_id % 100 == 0:
                print("epoch: {}, batch: {}, loss is: {}, acc is {}".format(epoch_id, batch_id, avg_loss.numpy(), acc.numpy()))
                log_writer.add_scalar(tag = 'acc_of_train', step = iter, value=acc.numpy() )
                log_writer.add_scalar(tag = 'loss_of_train', step = iter, value=avg_loss.numpy())
                iter += 100
            # 反向传播
            avg_loss.backward()
            opt.step()
            opt.clear_grad()
        
        for batch_id, data in enumerate(test_loader()):
            model.eval()
            images, labels = data
            images = paddle.to_tensor(images)
            labels = paddle.to_tensor(labels)

            # 预测
            predicts, acc = model(images, labels)
            loss = F.cross_entropy(predicts, labels)
            avg_loss = paddle.mean(loss)
            if batch_id % 100 == 0:
                log_writer.add_scalar(tag = 'acc_of_test', step=iter2, value=acc.numpy())
                log_writer.add_scalar(tag = 'loss_of_test', step=iter2, value=avg_loss.numpy())
                iter2 += 100


In [ ]:
model = MNIST_CONV_INIT()
train_MNIST_INIT(model)

In [9]:
paddle.jit.save(model, '.model', [paddle.static.InputSpec([-1,1,28,28]),None])

e:\SoftWare\Program\Anaconda\envs\d2l\lib\site-packages\paddle\fluid\variable_index.py:591: UserWarning: Warning: In Tensor '__getitem__', if the number of scalar elements in the index is equal to the rank of the Tensor, the output should be 0-D. In order to be consistent with the behavior of previous versions, it will be processed to 1-D. But it is not correct and will be removed in release 2.6. If 1-D is still wanted, please modify the index element from scalar to slice (e.g. 'x[i]' => 'x[i:i+1]').
  warnings.warn(


In [10]:
# 定义训练过程
from visualdl import LogWriter
log_writer = LogWriter(logdir="./log")
def train_MNIST_L2(model):

    # 定义优化器
    opt = paddle.optimizer.Adam(learning_rate=0.001, weight_decay=paddle.regularizer.L2Decay(coeff=1e-5),parameters=model.parameters())

    # 训练参数
    EPOCH_NUM = 5
    iter = 0
    iter2 = 0
    for epoch_id in range(EPOCH_NUM):
        for batch_id, data in enumerate(train_loader()):
            model.train()
            images, labels = data 
            images = paddle.to_tensor(images)
            labels = paddle.to_tensor(labels)

            # 前向
            predicts, acc = model(images, labels)

            # 损失
            loss = F.cross_entropy(predicts, labels)
            avg_loss = paddle.mean(loss)
            

            # 记录损失
            if batch_id % 100 == 0:
                print("epoch: {}, batch: {}, loss is: {}, acc is {}".format(epoch_id, batch_id, avg_loss.numpy(), acc.numpy()))
                log_writer.add_scalar(tag = 'acc_of_train_L2', step = iter, value=acc.numpy() )
                log_writer.add_scalar(tag = 'loss_of_train_L2', step = iter, value=avg_loss.numpy())
                iter += 100
            # 反向传播
            avg_loss.backward()
            opt.step()
            opt.clear_grad()
        
        for batch_id, data in enumerate(test_loader()):
            model.eval()
            images, labels = data
            images = paddle.to_tensor(images)
            labels = paddle.to_tensor(labels)

            # 预测
            predicts, acc = model(images, labels)
            loss = F.cross_entropy(predicts, labels)
            avg_loss = paddle.mean(loss)
            if batch_id % 100 == 0:
                log_writer.add_scalar(tag = 'acc_of_test_L2', step=iter2, value=acc.numpy())
                log_writer.add_scalar(tag = 'loss_of_test_L2', step=iter2, value=avg_loss.numpy())
                iter2 += 100

In [11]:
model = MNIST_CONV_INIT()
train_MNIST_L2(model)

epoch: 0, batch: 0, loss is: [3.888531], acc is [0.125]
epoch: 0, batch: 100, loss is: [0.21369022], acc is [0.90625]
epoch: 0, batch: 200, loss is: [0.25103122], acc is [0.921875]
epoch: 0, batch: 300, loss is: [0.10832155], acc is [0.953125]
epoch: 0, batch: 400, loss is: [0.03853818], acc is [1.]
epoch: 0, batch: 500, loss is: [0.06931215], acc is [0.984375]
epoch: 0, batch: 600, loss is: [0.16624391], acc is [0.953125]
epoch: 0, batch: 700, loss is: [0.23210882], acc is [0.9375]
epoch: 0, batch: 800, loss is: [0.20683259], acc is [0.9375]
epoch: 0, batch: 900, loss is: [0.14033127], acc is [0.96875]
epoch: 1, batch: 0, loss is: [0.02932457], acc is [1.]
epoch: 1, batch: 100, loss is: [0.0996362], acc is [0.984375]
epoch: 1, batch: 200, loss is: [0.1458218], acc is [0.953125]
epoch: 1, batch: 300, loss is: [0.05703544], acc is [0.96875]
epoch: 1, batch: 400, loss is: [0.0476568], acc is [0.984375]
epoch: 1, batch: 500, loss is: [0.02081344], acc is [1.]
epoch: 1, batch: 600, loss is

In [15]:
# 定义训练过程, 使用L1正则
from visualdl import LogWriter
log_writer = LogWriter(logdir="./log")
def train_MNIST_L1(model):

    # 定义优化器
    opt = paddle.optimizer.Adam(learning_rate=0.001, weight_decay=paddle.regularizer.L1Decay(coeff=1e-5),parameters=model.parameters())

    # 训练参数
    EPOCH_NUM = 5
    iter = 0
    iter2 = 0
    for epoch_id in range(EPOCH_NUM):
        for batch_id, data in enumerate(train_loader()):
            model.train()
            images, labels = data 
            images = paddle.to_tensor(images)
            labels = paddle.to_tensor(labels)

            # 前向
            predicts, acc = model(images, labels)

            # 损失
            loss = F.cross_entropy(predicts, labels)
            avg_loss = paddle.mean(loss)
            

            # 记录损失
            if batch_id % 100 == 0:
                print("epoch: {}, batch: {}, loss is: {}, acc is {}".format(epoch_id, batch_id, avg_loss.numpy(), acc.numpy()))
                log_writer.add_scalar(tag = 'acc_of_train_L1', step = iter, value=acc.numpy() )
                log_writer.add_scalar(tag = 'loss_of_train_L1', step = iter, value=avg_loss.numpy())
                iter += 100
            # 反向传播
            avg_loss.backward()
            opt.step()
            opt.clear_grad()
        
        for batch_id, data in enumerate(test_loader()):
            model.eval()
            images, labels = data
            images = paddle.to_tensor(images)
            labels = paddle.to_tensor(labels)

            # 预测
            predicts, acc = model(images, labels,model=)
            loss = F.cross_entropy(predicts, labels)
            avg_loss = paddle.mean(loss)
            if batch_id % 100 == 0:
                log_writer.add_scalar(tag = 'acc_of_test_L1', step=iter2, value=acc.numpy())
                log_writer.add_scalar(tag = 'loss_of_test_L1', step=iter2, value=avg_loss.numpy())
                iter2 += 100

In [ ]:
model_L1 = MNIST_CONV_INIT()
train_MNIST_L1(model_L1)

In [17]:
# 定义模型结构,dropout
import paddle.nn.functional as F
from paddle.nn import Conv2D, MaxPool2D, Linear

# 多层卷积神经网络实现
class MNIST_CONV_DROPOUT(paddle.nn.Layer):
    def __init__(self):
        super(MNIST_CONV_DROPOUT, self).__init__()

        # 定义卷积层，输出特征通道20，卷积核大小5
        self.conv1 = Conv2D(in_channels=1, out_channels=20, kernel_size=5, stride=1, padding=2)
        # 定义池化层，池化核的大小为2，步长为2
        self.max_pool1 = MaxPool2D(kernel_size=2, stride=2)
        # 定义卷积层，输出特征20维，卷积核大小为5
        self.conv2 = Conv2D(in_channels=20, out_channels=20, kernel_size=5, stride=1, padding=2)
        # 定义池化层
        self.max_pool2 = MaxPool2D(kernel_size=2, stride=2)
        # 定义一层全连接层，输入维度是10
        self.fc = Linear(in_features=980, out_features=10)

    # 定义前向，卷积后使用池化，然后全连接输出
    # 激活函数使用ReLU，全连接层使用softmax
    def forward(self, inputs, label,mode='upscale_in_train'):
        x = self.conv1(inputs)
        x = F.relu(x)
        x = self.max_pool1(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = self.max_pool2(x)
        x = paddle.reshape(x, [x.shape[0], 980])
        x = self.fc(x)

        # q全连接层加入一个dropout
        x = F.dropout(x, p = 0.01,mode=mode)
        if label is not None:
            acc = paddle.metric.accuracy(input=x, label=label)
            return x, acc
        else:
            return x
use_gpu = True
paddle.set_device('gpu:0') if use_gpu else paddle.set_device('cpu')



Place(gpu:0)

In [20]:
# 定义训练过程
from visualdl import LogWriter
log_writer = LogWriter(logdir="./log")
def train_MNIST_INIT(model):

    # 定义优化器
    opt = paddle.optimizer.Adam(learning_rate=0.003, parameters=model.parameters())

    # 训练参数
    EPOCH_NUM = 5
    iter = 0
    iter2 = 0
    for epoch_id in range(EPOCH_NUM):
        for batch_id, data in enumerate(train_loader()):
            model.train()
            images, labels = data 
            images = paddle.to_tensor(images)
            labels = paddle.to_tensor(labels)

            # 前向
            predicts, acc = model(images, labels)

            # 损失
            loss = F.cross_entropy(predicts, labels)
            avg_loss = paddle.mean(loss)
            

            # 记录损失
            if batch_id % 100 == 0:
                print("epoch: {}, batch: {}, loss is: {}, acc is {}".format(epoch_id, batch_id, avg_loss.numpy(), acc.numpy()))
                log_writer.add_scalar(tag = 'acc_of_train_dr1', step = iter, value=acc.numpy() )
                log_writer.add_scalar(tag = 'loss_of_train_dr1', step = iter, value=avg_loss.numpy())
                iter += 100
            # 反向传播
            avg_loss.backward()
            opt.step()
            opt.clear_grad()
        
        for batch_id, data in enumerate(test_loader()):
            model.eval()
            images, labels = data
            images = paddle.to_tensor(images)
            labels = paddle.to_tensor(labels)

            # 预测
            predicts, acc = model(images, labels,mode='downscale_in_infer')
            loss = F.cross_entropy(predicts, labels)
            avg_loss = paddle.mean(loss)
            if batch_id % 100 == 0:
                log_writer.add_scalar(tag = 'acc_of_test_dr1', step=iter2, value=acc.numpy())
                log_writer.add_scalar(tag = 'loss_of_test_dr1', step=iter2, value=avg_loss.numpy())
                iter2 += 100

In [21]:
model_dropout = MNIST_CONV_DROPOUT()
train_MNIST_INIT(model_dropout)

epoch: 0, batch: 0, loss is: [3.805146], acc is [0.125]
epoch: 0, batch: 100, loss is: [0.22334327], acc is [0.90625]
epoch: 0, batch: 200, loss is: [0.15081255], acc is [0.984375]
epoch: 0, batch: 300, loss is: [0.13811032], acc is [0.96875]
epoch: 0, batch: 400, loss is: [0.15092635], acc is [0.9375]
epoch: 0, batch: 500, loss is: [0.09098322], acc is [0.953125]
epoch: 0, batch: 600, loss is: [0.03913482], acc is [0.984375]
epoch: 0, batch: 700, loss is: [0.05470811], acc is [0.984375]
epoch: 0, batch: 800, loss is: [0.05528992], acc is [0.984375]
epoch: 0, batch: 900, loss is: [0.2042924], acc is [0.953125]
epoch: 1, batch: 0, loss is: [0.21055363], acc is [0.96875]
epoch: 1, batch: 100, loss is: [0.06998789], acc is [0.96875]
epoch: 1, batch: 200, loss is: [0.05319173], acc is [0.96875]
epoch: 1, batch: 300, loss is: [0.01125339], acc is [1.]
epoch: 1, batch: 400, loss is: [0.01823329], acc is [1.]
epoch: 1, batch: 500, loss is: [0.00940316], acc is [1.]
epoch: 1, batch: 600, loss 

In [ ]:
# 实现K折交叉验证，先定义两层卷积和一层全连接，增加一个dropout
# 定义模型结构,dropout
import paddle.nn.functional as F
from paddle.nn import Conv2D, MaxPool2D, Linear

# 多层卷积神经网络实现
class MNIST_CONV_DROPOUT(paddle.nn.Layer):
    def __init__(self):
        super(MNIST_CONV_DROPOUT, self).__init__()

        # 定义卷积层，输出特征通道20，卷积核大小5
        self.conv1 = Conv2D(in_channels=1, out_channels=20, kernel_size=5, stride=1, padding=2)
        # 定义池化层，池化核的大小为2，步长为2
        self.max_pool1 = MaxPool2D(kernel_size=2, stride=2)
        # 定义卷积层，输出特征20维，卷积核大小为5
        self.conv2 = Conv2D(in_channels=20, out_channels=20, kernel_size=5, stride=1, padding=2)
        # 定义池化层
        self.max_pool2 = MaxPool2D(kernel_size=2, stride=2)
        # 定义一层全连接层，输入维度是10
        self.fc = Linear(in_features=980, out_features=10)

    # 定义前向，卷积后使用池化，然后全连接输出
    # 激活函数使用ReLU，全连接层使用softmax
    def forward(self, inputs, label,mode='upscale_in_train'):
        x = self.conv1(inputs)
        x = F.relu(x)
        x = self.max_pool1(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = self.max_pool2(x)
        x = paddle.reshape(x, [x.shape[0], 980])
        x = self.fc(x)

        # q全连接层加入一个dropout
        x = F.dropout(x, p = 0.01,mode=mode)
        if label is not None:
            acc = paddle.metric.accuracy(input=x, label=label)
            return x, acc
        else:
            return x
use_gpu = True
paddle.set_device('gpu:0') if use_gpu else paddle.set_device('cpu')


In [34]:
import paddle
from paddle.vision.transforms import Normalize
transform = Normalize(mean=[127.5], std=[127.5], data_format='chw')
# 下载数据集并初始化 dataset
train_dataset = paddle.vision.datasets.MNIST(mode='train', transform=transform)
test_dataset = paddle.vision.datasets.MNIST(mode='test', transform=transform)

In [41]:
import numpy as np
def evaluation(model, datasets, mode='downscale_in_infer'):
    model.eval()
    acc_set = list()
    losses = []
    for batch_id, data in enumerate(datasets()):
        images, labels = data
        images = paddle.to_tensor(images)
        labels = paddle.to_tensor(labels)
        pred, ac = model(images,labels, mode=mode)   # 获取预测值
        loss = F.cross_entropy(pred, labels)
        avg_loss = paddle.mean(loss)
        acc = paddle.metric.accuracy(input=pred, label=labels)
        acc_set.extend(acc.numpy())
        losses.append(avg_loss)
    
    # #计算多个batch的准确率
    acc_val_mean = np.array(acc_set).mean()
    loss_val_mean = np.array(losses).mean()
    return acc_val_mean, loss_val_mean

In [46]:
from paddle.io import Subset
from visualdl import LogWriter
import paddle
import paddle.nn.functional as F
from sklearn.model_selection import KFold

# 定义要调整的超参数范围,我这里要搜索优化算法，batch大小
lr0 = 0
learning_rates = [0.001, 0.005, 0.009, 0.012]
batch_size = [32, 64, 128]
kf = KFold(n_splits=3, shuffle=True, random_state=42)

# 存储每个超参数组合的性能结果
results = []

log_writer = LogWriter(logdir="./log")

for lr in learning_rates:
    for b_s in batch_size:
        for fold, (train_indices, val_indices) in enumerate(kf.split(train_dataset), 1):
            print(f"Training on Fold {fold}")
            train_data = Subset(train_dataset, train_indices)
            val_data = Subset(train_dataset, val_indices)

            model = MNIST_CONV_DROPOUT()  # 创建模型
            opt = paddle.optimizer.Adam(learning_rate=lr, parameters=model.parameters())
            
            train_loader = paddle.io.DataLoader(train_data, batch_size=b_s, shuffle=True, num_workers=1, drop_last=True)
            EPOCH_NUM = 2
            for epoch in range(EPOCH_NUM):
                for batch_id, data in enumerate(train_loader()):
                    # 训练模型
                    images, labels = data 
                    images = paddle.to_tensor(images)
                    labels = paddle.to_tensor(labels)
                    
                    # 前向传播
                    predicts, ac = model(images, labels)
                    loss = F.cross_entropy(predicts, labels)
                    avg_loss = paddle.mean(loss)
                    
                    if batch_id % 200 ==0:
                        print(f'lr:{lr}, bitch_size:{b_s}, epoch:{epoch}, acc{ac.numpy()}, loss{avg_loss.numpy()}')
                    # 反向传播
                    avg_loss.backward()
                    opt.step()
                    opt.clear_grad()

                # 在验证集上评估性能
                val_loader = paddle.io.DataLoader(val_data, batch_size=b_s, shuffle=False, num_workers=1, drop_last=True)
                val_acc, val_loss = evaluation(model, val_loader)  # 自定义evaluate函数
                log_writer.add_scalar(tag='acc_of_val_1', step=(fold - 1) * EPOCH_NUM + epoch, value=val_acc)
                log_writer.add_scalar(tag='loss_of_val_1', step=(fold - 1) * EPOCH_NUM + epoch, value=val_loss)
            
            results.append((lr, b_s, val_acc))



Training on Fold 1
lr:0.001, bitch_size:32, epoch:0, acc[0.09375], loss[3.7911503]
lr:0.001, bitch_size:32, epoch:0, acc[0.9375], loss[0.1856969]
lr:0.001, bitch_size:32, epoch:0, acc[0.9375], loss[0.17303059]
lr:0.001, bitch_size:32, epoch:0, acc[0.875], loss[0.19910502]
lr:0.001, bitch_size:32, epoch:0, acc[0.96875], loss[0.11269292]
lr:0.001, bitch_size:32, epoch:0, acc[0.9375], loss[0.21490234]
lr:0.001, bitch_size:32, epoch:0, acc[0.96875], loss[0.07525603]
lr:0.001, bitch_size:32, epoch:1, acc[0.9375], loss[0.14679341]
lr:0.001, bitch_size:32, epoch:1, acc[1.], loss[0.03188223]
lr:0.001, bitch_size:32, epoch:1, acc[0.96875], loss[0.03552128]
lr:0.001, bitch_size:32, epoch:1, acc[0.96875], loss[0.0676665]
lr:0.001, bitch_size:32, epoch:1, acc[1.], loss[0.00960562]
lr:0.001, bitch_size:32, epoch:1, acc[0.96875], loss[0.11558908]
lr:0.001, bitch_size:32, epoch:1, acc[1.], loss[0.02667837]
Training on Fold 2
lr:0.001, bitch_size:32, epoch:0, acc[0.15625], loss[4.144415]
lr:0.001, bit

In [47]:
# 计算每个超参数组合的平均验证准确度
average_results = {}
for lr, b_s, _ in results:
    avg_val_acc = sum(val_acc for lr_, b_s_, val_acc in results if lr_ == lr and b_s_ == b_s) / 3
    average_results[(lr, b_s)] = avg_val_acc

# 选择具有最佳平均性能的超参数
best_lr, best_b_s = max(average_results, key=average_results.get)
best_val_acc = average_results[(best_lr, best_b_s)]

print(f"Best Learning Rate: {best_lr}")
print(f'Best batch size{best_b_s}')
print(f"Best Validation Accuracy: {best_val_acc}")



Best Learning Rate: 0.005
Best batch size128
Best Validation Accuracy: 0.9772469202677408


In [54]:
# 使用最佳超参数进行最终训练
# 定义训练过程
from visualdl import LogWriter
log_writer = LogWriter(logdir="./log")
def train_MNIST_INIT_k(model):

    # 定义优化器
    opt = paddle.optimizer.Adam(learning_rate=0.005, parameters=model.parameters())

    # 训练参数
    EPOCH_NUM = 5
    iter = 0
    iter2 = 0
    for epoch_id in range(EPOCH_NUM):
        for batch_id, data in enumerate(train_loader()):
            model.train()
            images, labels = data 
            images = paddle.to_tensor(images)
            labels = paddle.to_tensor(labels)

            # 前向
            predicts, acc = model(images, labels)

            # 损失
            loss = F.cross_entropy(predicts, labels)
            avg_loss = paddle.mean(loss)
            

            # 记录损失
            if batch_id % 100 == 0:
                print("epoch: {}, batch: {}, loss is: {}, acc is {}".format(epoch_id, batch_id, avg_loss.numpy(), acc.numpy()))
                log_writer.add_scalar(tag = 'acc_of_train_k', step = iter, value=acc.numpy() )
                log_writer.add_scalar(tag = 'loss_of_train_k', step = iter, value=avg_loss.numpy())
                iter += 100
            # 反向传播
            avg_loss.backward()
            opt.step()
            opt.clear_grad()
        
        for batch_id, data in enumerate(test_loader()):
            model.eval()
            images, labels = data
            images = paddle.to_tensor(images)
            labels = paddle.to_tensor(labels)

            # 预测
            predicts, acc = model(images, labels)
            loss = F.cross_entropy(predicts, labels)
            avg_loss = paddle.mean(loss)
            if batch_id % 100 == 0:
                log_writer.add_scalar(tag = 'acc_of_test_k', step=iter2, value=acc.numpy())
                log_writer.add_scalar(tag = 'loss_of_test_k', step=iter2, value=avg_loss.numpy())
                iter2 += 100

In [55]:
model_k = MNIST_CONV_DROPOUT()
train_MNIST_INIT_k(model_k)

epoch: 0, batch: 0, loss is: [3.07781], acc is [0.1484375]
epoch: 0, batch: 100, loss is: [0.36147293], acc is [0.890625]
epoch: 0, batch: 200, loss is: [0.24685222], acc is [0.9296875]
epoch: 0, batch: 300, loss is: [0.31525332], acc is [0.9140625]
epoch: 0, batch: 400, loss is: [0.12449379], acc is [0.9453125]
epoch: 1, batch: 0, loss is: [0.1812364], acc is [0.9296875]
epoch: 1, batch: 100, loss is: [0.21236974], acc is [0.9140625]
epoch: 1, batch: 200, loss is: [0.11446952], acc is [0.96875]
epoch: 1, batch: 300, loss is: [0.13346854], acc is [0.9609375]
epoch: 1, batch: 400, loss is: [0.09755027], acc is [0.9765625]
epoch: 2, batch: 0, loss is: [0.15984453], acc is [0.953125]
epoch: 2, batch: 100, loss is: [0.13029255], acc is [0.9609375]
epoch: 2, batch: 200, loss is: [0.09265825], acc is [0.9765625]
epoch: 2, batch: 300, loss is: [0.04480406], acc is [0.984375]
epoch: 2, batch: 400, loss is: [0.13464305], acc is [0.96875]
epoch: 3, batch: 0, loss is: [0.03072544], acc is [1.]
ep